<a href="https://colab.research.google.com/github/Gressling/digitalchemistry.org/blob/main/knowledge_graph_and_ontologies%5CKnowledge_Graph_and_Batch_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knowledge Graph and Batch Data

In [1]:
import sqlite3

# Create a connection to the Knowledge Graph database
conn_kg = sqlite3.connect('knowledge_graph.db')
cursor_kg = conn_kg.cursor()

# Create the reactions table
cursor_kg.execute("""
    CREATE TABLE reactions (
        id INTEGER PRIMARY KEY,
        reaction_name TEXT,
        catalyst TEXT,
        reactant TEXT,
        intermediate TEXT,
        product TEXT
    )
""")

# Insert the Nitration of Benzene reaction
cursor_kg.execute("""
    INSERT INTO reactions (reaction_name, catalyst, reactant, intermediate, product)
    VALUES ('Nitration of Benzene', 'Concentrated Sulfuric Acid', 'Benzene', 'Nitronium Ion', 'Nitrobenzene')
""")

conn_kg.commit()


In [2]:
# Create a connection to the Execution Layer database
conn_exec = sqlite3.connect('execution_layer.db')
cursor_exec = conn_exec.cursor()

# Create the inventory and batches tables
cursor_exec.execute("""
    CREATE TABLE inventory (
        id INTEGER PRIMARY KEY,
        chemical_name TEXT,
        quantity_liters REAL
    )
""")

cursor_exec.execute("""
    CREATE TABLE batches (
        id INTEGER PRIMARY KEY,
        reaction_id INTEGER,
        date_time TEXT,
        status TEXT,
        FOREIGN KEY (reaction_id) REFERENCES reactions (id)
    )
""")

# Insert some sample data
cursor_exec.execute("INSERT INTO inventory (chemical_name, quantity_liters) VALUES ('Benzene', 500.0)")
cursor_exec.execute("INSERT INTO inventory (chemical_name, quantity_liters) VALUES ('Concentrated Sulfuric Acid', 200.0)")
cursor_exec.execute("INSERT INTO batches (reaction_id, date_time, status) VALUES (1, '2023-08-17 10:00:00', 'Completed')")

conn_exec.commit()


In [3]:
# Query the reaction details from the Knowledge Graph
cursor_kg.execute("SELECT * FROM reactions WHERE reaction_name = 'Nitration of Benzene'")
reaction_details = cursor_kg.fetchone()

# Print the reaction details
reaction_id, reaction_name, catalyst, reactant, intermediate, product = reaction_details
print(f"Reaction Details:\nReaction: {reaction_name}\nCatalyst: {catalyst}\nReactant: {reactant}\nIntermediate: {intermediate}\nProduct: {product}\n")

# Query the corresponding batch processing records from the Execution Layer using the reaction_id
cursor_exec.execute("SELECT date_time, status FROM batches WHERE reaction_id = ?", (reaction_id,))
batch_records = cursor_exec.fetchall()

# Print the batch processing records
print("Batch Processing Records:")
for batch_record in batch_records:
    date_time, status = batch_record
    print(f"Date/Time: {date_time}\nStatus: {status}\n")


Reaction Details:
Reaction: Nitration of Benzene
Catalyst: Concentrated Sulfuric Acid
Reactant: Benzene
Intermediate: Nitronium Ion
Product: Nitrobenzene

Batch Processing Records:
Date/Time: 2023-08-17 10:00:00
Status: Completed



In [4]:
# Insert additional batch records for the Nitration of Benzene reaction
cursor_exec.execute("INSERT INTO batches (reaction_id, date_time, status) VALUES (1, '2023-08-18 08:30:00', 'In Progress')")
cursor_exec.execute("INSERT INTO batches (reaction_id, date_time, status) VALUES (1, '2023-08-15 14:00:00', 'Completed')")
cursor_exec.execute("INSERT INTO batches (reaction_id, date_time, status) VALUES (1, '2023-08-16 12:00:00', 'Failed')")

conn_exec.commit()


In [5]:
# Query the reaction details from the Knowledge Graph
cursor_kg.execute("SELECT * FROM reactions WHERE reaction_name = 'Nitration of Benzene'")
reaction_details = cursor_kg.fetchone()

# Print the reaction details
reaction_id, reaction_name, catalyst, reactant, intermediate, product = reaction_details
print(f"Reaction Details:\nReaction: {reaction_name}\nCatalyst: {catalyst}\nReactant: {reactant}\nIntermediate: {intermediate}\nProduct: {product}\n")

# Query the corresponding batch processing records from the Execution Layer using the reaction_id
cursor_exec.execute("SELECT date_time, status FROM batches WHERE reaction_id = ?", (reaction_id,))
batch_records = cursor_exec.fetchall()

# Print the batch processing records
print("Batch Processing Records:")
for batch_record in batch_records:
    date_time, status = batch_record
    print(f"Date/Time: {date_time}\nStatus: {status}\n")


Reaction Details:
Reaction: Nitration of Benzene
Catalyst: Concentrated Sulfuric Acid
Reactant: Benzene
Intermediate: Nitronium Ion
Product: Nitrobenzene

Batch Processing Records:
Date/Time: 2023-08-17 10:00:00
Status: Completed

Date/Time: 2023-08-18 08:30:00
Status: In Progress

Date/Time: 2023-08-15 14:00:00
Status: Completed

Date/Time: 2023-08-16 12:00:00
Status: Failed



In [6]:
# Add the expected_yield column to the reactions table
cursor_kg.execute("ALTER TABLE reactions ADD COLUMN expected_yield REAL")

# Update the Nitration of Benzene reaction with an expected yield of 90%
cursor_kg.execute("UPDATE reactions SET expected_yield = ? WHERE reaction_name = ?", (90.0, 'Nitration of Benzene'))

conn_kg.commit()


In [7]:
# Add the actual_yield column to the batches table
cursor_exec.execute("ALTER TABLE batches ADD COLUMN actual_yield REAL")


In [8]:
# Update the existing batch records with actual yields
cursor_exec.execute("UPDATE batches SET actual_yield = ? WHERE date_time = ?", (91.0, '2023-08-17 10:00:00'))
cursor_exec.execute("UPDATE batches SET actual_yield = ? WHERE date_time = ?", (85.0, '2023-08-15 14:00:00'))
cursor_exec.execute("UPDATE batches SET actual_yield = ? WHERE date_time = ?", (75.0, '2023-08-18 08:30:00'))
cursor_exec.execute("UPDATE batches SET actual_yield = ? WHERE date_time = ?", (93.0, '2023-08-16 12:00:00'))

conn_exec.commit()

In [9]:
# Query the expected yield from the Knowledge Graph
cursor_kg.execute("SELECT expected_yield FROM reactions WHERE reaction_name = 'Nitration of Benzene'")
expected_yield = cursor_kg.fetchone()[0]

# Query the actual yields from the Execution Layer
cursor_exec.execute("SELECT date_time, actual_yield FROM batches WHERE reaction_id = ?", (reaction_id,))
batch_yields = cursor_exec.fetchall()

# Analyze and compare the yields
print(f"Expected Yield for Nitration of Benzene: {expected_yield}%\n")
print("Batch Yields:")
for batch_yield in batch_yields:
    date_time, actual_yield = batch_yield
    deviation = actual_yield - expected_yield
    print(f"Date/Time: {date_time}\nActual Yield: {actual_yield}%\nDeviation from Expected: {deviation}%\n")


Expected Yield for Nitration of Benzene: 90.0%

Batch Yields:
Date/Time: 2023-08-17 10:00:00
Actual Yield: 91.0%
Deviation from Expected: 1.0%

Date/Time: 2023-08-18 08:30:00
Actual Yield: 75.0%
Deviation from Expected: -15.0%

Date/Time: 2023-08-15 14:00:00
Actual Yield: 85.0%
Deviation from Expected: -5.0%

Date/Time: 2023-08-16 12:00:00
Actual Yield: 93.0%
Deviation from Expected: 3.0%



In [10]:
# Insert 10 batch records for the Nitration of Benzene reaction (two with lower yield)
actual_yields = [88, 91, 89, 92, 85, 90, 91, 93, 89, 87] # Two yields (85, 87) do not meet the expected yield
for i, actual_yield in enumerate(actual_yields):
    cursor_exec.execute("INSERT INTO batches (reaction_id, date_time, status, actual_yield) VALUES (1, ?, 'Completed', ?)", (f'2023-08-1{i+1} 10:00:00', actual_yield))

conn_exec.commit()

In [11]:
# Query the expected yield from the Knowledge Graph
cursor_kg.execute("SELECT expected_yield FROM reactions WHERE reaction_name = 'Nitration of Benzene'")
expected_yield = cursor_kg.fetchone()[0]

# Query the actual yields from the Execution Layer
cursor_exec.execute("SELECT date_time, actual_yield FROM batches WHERE reaction_id = 1")
batch_yields = cursor_exec.fetchall()

# Analyze and compare the yields
print(f"Expected Yield for Nitration of Benzene: {expected_yield}%\n")
print("Batch Yields:")
for batch_yield in batch_yields:
    date_time, actual_yield = batch_yield
    deviation = actual_yield - expected_yield
    print(f"Date/Time: {date_time}\nActual Yield: {actual_yield}%\nDeviation from Expected: {deviation}%\n")


Expected Yield for Nitration of Benzene: 90.0%

Batch Yields:
Date/Time: 2023-08-17 10:00:00
Actual Yield: 91.0%
Deviation from Expected: 1.0%

Date/Time: 2023-08-18 08:30:00
Actual Yield: 75.0%
Deviation from Expected: -15.0%

Date/Time: 2023-08-15 14:00:00
Actual Yield: 85.0%
Deviation from Expected: -5.0%

Date/Time: 2023-08-16 12:00:00
Actual Yield: 93.0%
Deviation from Expected: 3.0%

Date/Time: 2023-08-11 10:00:00
Actual Yield: 88.0%
Deviation from Expected: -2.0%

Date/Time: 2023-08-12 10:00:00
Actual Yield: 91.0%
Deviation from Expected: 1.0%

Date/Time: 2023-08-13 10:00:00
Actual Yield: 89.0%
Deviation from Expected: -1.0%

Date/Time: 2023-08-14 10:00:00
Actual Yield: 92.0%
Deviation from Expected: 2.0%

Date/Time: 2023-08-15 10:00:00
Actual Yield: 85.0%
Deviation from Expected: -5.0%

Date/Time: 2023-08-16 10:00:00
Actual Yield: 90.0%
Deviation from Expected: 0.0%

Date/Time: 2023-08-17 10:00:00
Actual Yield: 91.0%
Deviation from Expected: 1.0%

Date/Time: 2023-08-18 10:00:00